# Модуль А

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

## 1.1 Загрузка данных

In [2]:
file = pd.read_csv('/'.join(os.getcwd().split('/')[:-1]) + '/data/brooklyn_sales_map.csv', index_col=0)
file

/Users/pavel/Desktop/DS2021_Qual/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (40,41,43,45,46,47,86) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,borough,neighborhood,building_class_category,tax_class,block,lot,easement,building_class,address,apartment_number,...,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLA,PFIRM15_FL,Version,MAPPLUTO_F,SHAPE_Leng,SHAPE_Area
1,3,DOWNTOWN-METROTECH,28 COMMERCIAL CONDOS,4,140,1001,NaN,R5,330 JAY STREET,COURT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,DOWNTOWN-FULTON FERRY,29 COMMERCIAL GARAGES,4,54,1,NaN,G7,85 JAY STREET,NaN,...,NaN,3.000540e+09,12/06/2002,1.0,NaN,NaN,17V1.1,0.0,1559.889144,140131.577176
3,3,BROOKLYN HEIGHTS,21 OFFICE BUILDINGS,4,204,1,NaN,O6,29 COLUMBIA HEIGHTS,NaN,...,NaN,0.000000e+00,NaN,1.0,NaN,NaN,17V1.1,0.0,890.718521,34656.447240
4,3,MILL BASIN,22 STORE BUILDINGS,4,8470,55,NaN,K6,5120 AVENUE U,NaN,...,NaN,0.000000e+00,NaN,1.0,1.0,1.0,17V1.1,0.0,3729.786857,797554.847834
5,3,BROOKLYN HEIGHTS,26 OTHER HOTELS,4,230,1,NaN,H8,21 CLARK STREET,NaN,...,NaN,0.000000e+00,NaN,1.0,NaN,NaN,17V1.1,0.0,620.761169,21360.147631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390879,3,WYCKOFF HEIGHTS,31 COMMERCIAL VACANT LAND,1,3337,19,NaN,C0,364 LINDEN STREET,NaN,...,NaN,0.000000e+00,NaN,1.0,NaN,NaN,17V1.1,0.0,244.422014,2501.088819
390880,3,WYCKOFF HEIGHTS,31 COMMERCIAL VACANT LAND,1,3337,19,NaN,C0,364 LINDEN STREET,NaN,...,NaN,0.000000e+00,NaN,1.0,NaN,NaN,17V1.1,0.0,244.422014,2501.088819
390881,3,WYCKOFF HEIGHTS,41 TAX CLASS 4 - OTHER,4,3280,50,NaN,Z9,383 HARMAN STREET,NaN,...,NaN,0.000000e+00,NaN,1.0,NaN,NaN,17V1.1,0.0,231.409296,2795.673780
390882,3,3004,04 TAX CLASS 1 CONDOS,NaN,6886,1107,NaN,NaN,14 BAY 47TH STREET,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Судя по описанию данных, техническими являются следующие поля:
- Version
- PLUTOMapID
- EDesigNum
- TaxMap
- Sanborn
- ZMCode
- ZoneMap
- XCoord
- YCoord
- Tract2010
- CT2010
- CB2010
- BBL
- AreaSource
- MAPPLUTO_F
- SHAPE_Leng
- SHAPE_Area
- Address

Удалим эти данные

In [3]:
to_delete = ['Version',
             'PLUTOMapID',
             'EDesigNum',
             'EDesigNum',
             'Sanborn',
             'ZMCode',
             'ZoneMap',
             'XCoord',
             'YCoord',
             'Tract2010',
             'CT2010',
             'CB2010',
             'BBL',
             'AreaSource',
             'Address',
             'MAPPLUTO_F',
             'SHAPE_Area',
             'SHAPE_Leng']
file = file.drop(columns=to_delete)

## 1.2 Предобработка данных

Посчитаем в столбцах количество полей NaN. Если таких полей больше некоторого количества, то их следует удалить. Думаю, что убрать столбцы с 50% NaN и больше будет логично, потому что восстановление закономерностей в таких cтолбцах - очень сложный и долгий процесс.

In [4]:
dictio = dict(file.isna().sum())
threshold = 0.5
for i in dictio:
    if dictio[i]/len(file[i]) > threshold:
        file = file.drop(columns=i)
file

,borough,neighborhood,building_class_category,tax_class,block,lot,building_class,address,zip_code,residential_units,...,YearAlter1,YearAlter2,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,CondoNo,TaxMap,APPBBL
1,3,DOWNTOWN-METROTECH,28 COMMERCIAL CONDOS,4,140,1001,R5,330 JAY STREET,11201,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,DOWNTOWN-FULTON FERRY,29 COMMERCIAL GARAGES,4,54,1,G7,85 JAY STREET,11201,0,...,0.0,0.0,0.00,7.20,2.0,6.5,3.0,0.0,30101.0,3.000540e+09
3,3,BROOKLYN HEIGHTS,21 OFFICE BUILDINGS,4,204,1,O6,29 COLUMBIA HEIGHTS,11201,0,...,1980.0,0.0,9.52,0.00,2.0,0.0,3.0,0.0,30106.0,0.000000e+00
4,3,MILL BASIN,22 STORE BUILDINGS,4,8470,55,K6,5120 AVENUE U,11234,0,...,0.0,0.0,2.82,0.00,2.0,0.0,3.0,0.0,32502.0,0.000000e+00
5,3,BROOKLYN HEIGHTS,26 OTHER HOTELS,4,230,1,H8,21 CLARK STREET,11201,0,...,1998.0,2000.0,17.57,2.43,0.0,4.8,3.0,0.0,30106.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390879,3,WYCKOFF HEIGHTS,31 COMMERCIAL VACANT LAND,1,3337,19,C0,364 LINDEN STREET,11237,3,...,0.0,0.0,1.70,2.43,0.0,4.8,3.0,0.0,31106.0,0.000000e+00
390880,3,WYCKOFF HEIGHTS,31 COMMERCIAL VACANT LAND,1,3337,19,C0,364 LINDEN STREET,11237,3,...,0.0,0.0,1.70,2.43,0.0,4.8,3.0,0.0,31106.0,0.000000e+00
390881,3,WYCKOFF HEIGHTS,41 TAX CLASS 4 - OTHER,4,3280,50,Z9,383 HARMAN STREET,11237,0,...,0.0,0.0,0.00,2.43,0.0,4.8,3.0,0.0,31104.0,0.000000e+00
390882,3,3004,04 TAX CLASS 1 CONDOS,NaN,6886,1107,NaN,14 BAY 47TH STREET,11214,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Закодируем оставшиеся поля типа Object с помощью ```LabelEncoder``` из ```sklearn.preprocessing```

In [5]:
to_encode = file.select_dtypes(include=['object']).columns
cnt = 1
for i in to_encode:
    print(f"{cnt}/{len(to_encode)} - {i}")
    if i != 'building_class_category':
        file[i] = LabelEncoder().fit_transform(file[i])
    cnt += 1
file

1/15 - neighborhood
2/15 - building_class_category
3/15 - tax_class
4/15 - building_class
5/15 - address
6/15 - building_class_at_sale
7/15 - sale_date
8/15 - Borough
9/15 - FireComp
10/15 - SanitSub
11/15 - ZoneDist1
12/15 - SplitZone
13/15 - BldgClass
14/15 - OwnerName
15/15 - IrrLotCode


,borough,neighborhood,building_class_category,tax_class,block,lot,building_class,address,zip_code,residential_units,...,YearAlter1,YearAlter2,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,CondoNo,TaxMap,APPBBL
1,3,25,28 COMMERCIAL CONDOS,9,140,1001,128,130856,11201,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,23,29 COMMERCIAL GARAGES,9,54,1,52,208471,11201,0,...,0.0,0.0,0.00,7.20,2.0,6.5,3.0,0.0,30101.0,3.000540e+09
3,3,10,21 OFFICE BUILDINGS,9,204,1,104,119217,11201,0,...,1980.0,0.0,9.52,0.00,2.0,0.0,3.0,0.0,30106.0,0.000000e+00
4,3,44,22 STORE BUILDINGS,9,8470,55,81,164330,11234,0,...,0.0,0.0,2.82,0.00,2.0,0.0,3.0,0.0,32502.0,0.000000e+00
5,3,10,26 OTHER HOTELS,9,230,1,62,84851,11201,0,...,1998.0,2000.0,17.57,2.43,0.0,4.8,3.0,0.0,30106.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390879,3,63,31 COMMERCIAL VACANT LAND,0,3337,19,13,138138,11237,3,...,0.0,0.0,1.70,2.43,0.0,4.8,3.0,0.0,31106.0,0.000000e+00
390880,3,63,31 COMMERCIAL VACANT LAND,0,3337,19,13,138138,11237,3,...,0.0,0.0,1.70,2.43,0.0,4.8,3.0,0.0,31106.0,0.000000e+00
390881,3,63,41 TAX CLASS 4 - OTHER,9,3280,50,182,141531,11237,0,...,0.0,0.0,0.00,2.43,0.0,4.8,3.0,0.0,31104.0,0.000000e+00
390882,3,0,04 TAX CLASS 1 CONDOS,10,6886,1107,183,35223,11214,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Данные в столбце building_class_category иногда принимают одинаковые значения, различающиеся только в количестве пробелов. Например, возможна такая ситуация:
- ```04  TAX CLASS 1 CONDOS```
- ```04 TAX CLASS 1 CONDOS```

Эти классы, хотя и являются одинаковыми, не будут считаться таковыми при обработке, что может негативным образом повлиять на результаты работы. Обработаем эти данные, убрав лишниее пробелы.

In [6]:
copy_data = list(file['building_class_category'])
for i in range(len(copy_data)):
    if copy_data[i]:
        try:
            copy_data[i] = ' '.join(copy_data[i].split())
        except:
            pass
file['building_class_category'].update(copy_data)

Отдельно закодируем столбец ```building_class_category```, чтобы сохранить классы, присвоенные каждой группе

In [7]:
encode_bcc = LabelEncoder()
file['building_class_category'] = encode_bcc.fit_transform(file['building_class_category'])
file

,borough,neighborhood,building_class_category,tax_class,block,lot,building_class,address,zip_code,residential_units,...,YearAlter1,YearAlter2,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,CondoNo,TaxMap,APPBBL
1,3,25,37,9,140,1001,128,130856,11201,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,23,27,9,54,1,52,208471,11201,0,...,0.0,0.0,0.00,7.20,2.0,6.5,3.0,0.0,30101.0,3.000540e+09
3,3,10,29,9,204,1,104,119217,11201,0,...,1980.0,0.0,9.52,0.00,2.0,0.0,3.0,0.0,30106.0,0.000000e+00
4,3,44,32,9,8470,55,81,164330,11234,0,...,0.0,0.0,2.82,0.00,2.0,0.0,3.0,0.0,32502.0,0.000000e+00
5,3,10,40,9,230,1,62,84851,11201,0,...,1998.0,2000.0,17.57,2.43,0.0,4.8,3.0,0.0,30106.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390879,3,63,40,0,3337,19,13,138138,11237,3,...,0.0,0.0,1.70,2.43,0.0,4.8,3.0,0.0,31106.0,0.000000e+00
390880,3,63,56,0,3337,19,13,138138,11237,3,...,0.0,0.0,1.70,2.43,0.0,4.8,3.0,0.0,31106.0,0.000000e+00
390881,3,63,4,9,3280,50,182,141531,11237,0,...,0.0,0.0,0.00,2.43,0.0,4.8,3.0,0.0,31104.0,0.000000e+00
390882,3,0,4,10,6886,1107,183,35223,11214,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Сбросим эти данные в JSON файл, чтобы загрузить потом, если понадобится

In [8]:
import json


with open("classes.json", "w") as dump_file:
    json.dump(dict(zip(range(len(encode_bcc.classes_)), encode_bcc.classes_)), dump_file)
    

Следующий шаг - подстановка nan значений. Очевидным шагом будет использование ```Imputer``` из пакета ```sklearn.impute```. ```SimpleImputer``` не очень подходит под наши цели, потому что его использование может уничтожить некоторые закономерности.
Хорошим вариантом будет использование либо ```IterativeImputer```, либо ```KNNImputer```. Первый, по сути, является алгоритмом, подбирающим нужное значение, а второй берет значение на основе анализа похожих данных (т.е. имеющих схожие координаты в n-мерном векторном пространстве). Лучше будет воспользоваться первым методом, потому что метод K-ближайших соседей лучше всего работает на небольшом количестве измерений пространства.

In [9]:
imputer = IterativeImputer(random_state=42,
                           initial_strategy='median',
                           skip_complete=True,
                           tol=1e-2,
                           max_iter=15,
                           verbose=2)
file_iter_imputed = imputer.fit_transform(file)

[IterativeImputer] Completing matrix with shape (390883, 75)
[IterativeImputer] Ending imputation round 1/15, elapsed time 138.70
[IterativeImputer] Change: 7745740268.230192, scaled tolerance: 30893207.82 
[IterativeImputer] Ending imputation round 2/15, elapsed time 278.11
[IterativeImputer] Change: 6306199151.248339, scaled tolerance: 30893207.82 
[IterativeImputer] Ending imputation round 3/15, elapsed time 418.24
[IterativeImputer] Change: 1394713525.92834, scaled tolerance: 30893207.82 
[IterativeImputer] Ending imputation round 4/15, elapsed time 555.58
[IterativeImputer] Change: 444482876.6448251, scaled tolerance: 30893207.82 
[IterativeImputer] Ending imputation round 5/15, elapsed time 694.65
[IterativeImputer] Change: 622585464.1796839, scaled tolerance: 30893207.82 
[IterativeImputer] Ending imputation round 6/15, elapsed time 835.13
[IterativeImputer] Change: 540592878.6085109, scaled tolerance: 30893207.82 
[IterativeImputer] Ending imputation round 7/15, elapsed time 97

/Users/pavel/Desktop/DS2021_Qual/venv/lib/python3.9/site-packages/sklearn/impute/_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


Масштабируем данные для дальнейшего использования. Воспользуемся ```RobustScaler``` из ```sklearn.preprocessing```, который устойчив к выбросам, которые могут наблюдаться в датасете.

In [15]:
transformer = RobustScaler()
to_save = pd.DataFrame(file_iter_imputed, index=file.index, columns=file.columns).building_class_category
file_transformed = pd.DataFrame(transformer.fit_transform(file_iter_imputed), index = file.index, columns=file.columns)
file_transformed.building_class_category.update(to_save)

,borough,neighborhood,building_class_category,tax_class,block,lot,building_class,address,zip_code,residential_units,...,YearAlter1,YearAlter2,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,CondoNo,TaxMap,APPBBL
1,0.0,-0.060606,37.0,2.25,-0.867889,13.095890,0.982609,0.118699,-0.857143,-1.0,...,9.905089,1182.411705,8.891616,3.341130,6.335205,1.541960,0.0,16.017166,-1.139996,27.665151
2,0.0,-0.121212,27.0,2.25,-0.885810,-0.602740,0.321739,0.805199,-0.857143,-1.0,...,0.000000,0.000000,-0.839415,2.091041,2.000000,1.288287,0.0,0.000000,-0.894741,17.339153
3,0.0,-0.515152,29.0,2.25,-0.854553,-0.602740,0.773913,0.015753,-0.857143,-1.0,...,3.465208,0.000000,4.488071,-0.804246,2.000000,-1.104246,0.0,0.000000,-0.891032,0.000000
4,0.0,0.515152,32.0,2.25,0.867889,0.136986,0.573913,0.414775,0.714286,-1.0,...,0.000000,0.000000,0.738685,-0.804246,2.000000,-1.104246,0.0,0.000000,0.886580,0.000000
5,0.0,-0.515152,40.0,2.25,-0.849135,-0.602740,0.408696,-0.288212,-0.857143,-1.0,...,3.496710,2000.000000,8.992930,0.172913,0.000000,0.662548,0.0,0.000000,-0.891032,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390879,0.0,1.090909,40.0,0.00,-0.201709,-0.356164,-0.017391,0.183108,0.857143,2.0,...,0.000000,0.000000,0.111922,0.172913,0.000000,0.662548,0.0,0.000000,-0.149124,0.000000
390880,0.0,1.090909,56.0,0.00,-0.201709,-0.356164,-0.017391,0.183108,0.857143,2.0,...,0.000000,0.000000,0.111922,0.172913,0.000000,0.662548,0.0,0.000000,-0.149124,0.000000
390881,0.0,1.090909,4.0,2.25,-0.213586,0.068493,1.452174,0.213119,0.857143,-1.0,...,0.000000,0.000000,-0.839415,0.172913,0.000000,0.662548,0.0,0.000000,-0.150607,0.000000
390882,0.0,-0.818182,4.0,2.50,0.537820,14.547945,1.460870,-0.727169,-0.238095,-1.0,...,1.289024,-16.101045,0.912040,0.822468,-0.342939,0.262835,0.0,17.698443,0.344736,2.700453


Сохраним полученные данные в ```.csv``` файл

In [16]:
file_transformed.to_csv('file_transformed.csv')